In [59]:
import sys
import os
sys.path.append(os.getenv("PYTHONPATH", "/app")) #REVIEW
from utils import Utils

In [60]:
utils = Utils()
spark = utils.get_spark_session()

In [61]:
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [62]:
# The schema parameter is not mandatory. But not passing this info will generate a dataframe with indexes as columns and the types will be automatically inserted
emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [63]:
# Prints the schema in a readable, tree-like format. Displays column names, data types, and nullability.
emp.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [64]:
# Returns the schema as a programmatic StructType object.
emp.schema

StructType([StructField('employee_id', StringType(), True), StructField('department_id', StringType(), True), StructField('name', StringType(), True), StructField('age', StringType(), True), StructField('gender', StringType(), True), StructField('salary', StringType(), True), StructField('hire_date', StringType(), True)])

In [65]:
# Spark stores the schema in a list of StructType, with each field being a StructField()
# Defining a StructType schema:
from pyspark.sql.types import StructType, StructField, StringType

schema_spark = StructType([
    StructField("employee_id", StringType(), True), # This third parameter is the 'nullable' (defaults to True)
    StructField("department_id", StringType()),
    StructField("name", StringType()),
    StructField("age", StringType()),
    StructField("gender", StringType()),
    StructField("salary", StringType()),
    StructField("hire_date", StringType())
])

# The string schema ("employee_id string, department_id string, ..." can also be used to define the spark schema. Spark implicitly converts the schema string into the spark native data language

In [66]:
# Get columns
from pyspark.sql.functions import col
print(col("employee_id"))
print(emp.name)
print(emp["department_id"])

Column<'employee_id'>
Column<'name'>
Column<'department_id'>


In [ ]:
# Selecting columns:
# Column names are expressions
emp.select("age", emp.name, col("employee_id")).show()
# Select is a transformation, show is the action

In [ ]:
# The function expr() allows for expression evaluation when transforming columns
from pyspark.sql.functions import expr
filtered_emp = emp.select(expr("employee_id as employee"), "name", expr("cast (age as int) as idade"))
filtered_emp.show()
print(filtered_emp.schema)

In [ ]:
# The function selectExpr() allows for expresion evaluation in a select() without calling the expr() function
# Native function
emp.selectExpr("employee_id as employee", "name", "cast (age as int) as idade").show()

In [ ]:
# The where function is used to filter data through an expression
emp.select("name", "employee_id", "gender", "salary").where("age > 30").show()

In [ ]:
emp.select("name", "employee_id", "gender", "salary").where("age > 30").write.format("csv").save("/app/outputs/2/emp.csv")

In [57]:
# Spark converts the string schema into StructType via the private function parse_datatype_string:
from pyspark.sql.types import _parse_datatype_string

schema_str = "name string, age int, salary float"

_parse_datatype_string(schema_str)

StructType([StructField('name', StringType(), True), StructField('age', IntegerType(), True), StructField('salary', FloatType(), True)])